## ☕ Data Cleaning & Preparation

### 📥 1. Data Loading
- Imported the required libraries: **pandas** and **NumPy**  

In [15]:
import pandas as pd
import numpy as np

- Loaded the dataset into a pandas DataFrame  

In [4]:
df = pd.read_csv(r'C:\Users\Anubhav\OneDrive\Desktop\Portfolio\dirty_cafe_sales.csv')

### 🔎 2. Data Assessment & Understanding
- Computed the total number of null values across all columns  

In [30]:
df.isnull().sum()

Transaction ID         0
Item                 969
Quantity              38
Price Per Unit        38
Total Spent           40
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

- Generated a quick statistical summary of the dataset  
- Observed that there were no duplicate transactions, as each **Transaction ID** had a frequency of 1  
- Identified that the dataset contains **10 unique items**  

In [8]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


- Noted that all columns were stored as **object data types**, indicating the need for appropriate type conversions  

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


### ⚠️ 3. Data Quality Assessment
- Calculated both the **count and percentage of missing values** for each column to evaluate data completeness   

In [10]:
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [13]:
df.isnull().mean()*100

Transaction ID       0.00
Item                 3.33
Quantity             1.38
Price Per Unit       1.79
Total Spent          1.73
Payment Method      25.79
Location            32.65
Transaction Date     1.59
dtype: float64

- Identified ambiguous or irregular placeholders **ERROR** and **UNKNOWN** markers present in the dataset  
- Replaced these placeholders with proper missing values (**NaN**), which increased the null count but improved data accuracy and consistency 

In [16]:
df.replace(['ERROR', 'UNKNOWN'], np.nan, inplace=True)

In [17]:
df.isnull().sum()

Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

### 🧹 4. Data Cleaning — Type Conversion
Converted columns to appropriate data types:

- **Numeric:** Quantity, Price Per Unit, Total Spent  
- **Datetime:** Transaction Date  
- **String:** Transaction ID, Item, Payment Method, Location  

In [19]:
numeric_cols = ['Quantity', 'Price Per Unit', 'Total Spent']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

text_cols = ['Transaction ID', 'Item', 'Payment Method', 'Location']

for col in text_cols:
    df[col] = df[col].astype('string')

- Verified the conversions using the DataFrame information command to confirm updated types and non-null counts  

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  string        
 1   Item              9031 non-null   string        
 2   Quantity          9521 non-null   float64       
 3   Price Per Unit    9467 non-null   float64       
 4   Total Spent       9498 non-null   float64       
 5   Payment Method    6822 non-null   string        
 6   Location          6039 non-null   string        
 7   Transaction Date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), string(4)
memory usage: 625.1 KB


### 🧮 5. Data Cleaning — Numerical Imputation (Business Logic)

Missing values in key financial fields were imputed using domain-based calculations:

- **Quantity** was imputed using:  
  `Quantity = Total Spent ÷ Price Per Unit`  
  (Applied only when both Total Spent and Price Per Unit were available)

In [24]:
mask = df['Quantity'].isna() & df['Price Per Unit'].notna() & df['Total Spent'].notna()
df.loc[mask, 'Quantity'] = df.loc[mask, 'Total Spent'] / df.loc[mask, 'Price Per Unit']

- **Price Per Unit** was imputed using:  
  `Price Per Unit = Total Spent ÷ Quantity`  
  (Applied only when both Total Spent and Quantity were available)

In [26]:
mask = df['Price Per Unit'].isna() & df['Quantity'].notna() & df['Total Spent'].notna()
df.loc[mask, 'Price Per Unit'] = df.loc[mask, 'Total Spent'] / df.loc[mask, 'Quantity']

- **Total Spent** was imputed using:  
  `Total Spent = Quantity × Price Per Unit`  
  (Applied only when both Quantity and Price Per Unit were available)

In [28]:
mask = df['Total Spent'].isna() & df['Quantity'].notna() & df['Price Per Unit'].notna()
df.loc[mask, 'Total Spent'] = df.loc[mask, 'Quantity'] * df.loc[mask, 'Price Per Unit']

- These operations significantly reduced missing values in all three financial columns 

In [31]:
df.isnull().sum()

Transaction ID         0
Item                 969
Quantity              38
Price Per Unit        38
Total Spent           40
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

### 🏷️ 6. Data Cleaning — Categorical Imputation

Missing values in categorical fields were replaced with explicit labels to preserve records while maintaining transparency:

- Item → **"Unknown Item"**  
- Location → **"Unknown Location"**  
- Payment Method → **"Unknown Payment Method"**  
- Transaction Date → **"Unknown Date"**  

In [40]:
df['Item'] = df['Item'].fillna('Unknown Item')
df['Location'] = df['Location'].fillna('Unknown Location')
df['Payment Method'] = df['Payment Method'].fillna('Unknown Payment')
df['Transaction Date'] = df['Transaction Date'].fillna('Unknown Date')

### 🧩 7. Analysis of Remaining Missing Financial Records
- Conducted a detailed statistical analysis of records where **Total Spent** remained missing  
- Identified only **40 such records**, with just **20 containing non-missing values** for Quantity and Price Per Unit  
- Due to insufficient information to reliably reconstruct transaction values, these records were deemed unusable for revenue analysis  

In [42]:
df[df['Total Spent'].isna()].describe(include='all')

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,40,40,20.000000,20.00000,0.0,40,40,40
unique,40,9,NaN,NaN,NaN,4,3,39
top,TXN_4987129,Sandwich,NaN,NaN,NaN,Unknown Payment,In-store,2023-01-18 00:00:00
freq,1,8,NaN,NaN,NaN,14,14,2
mean,NaN,NaN,2.950000,2.97500,NaN,NaN,NaN,NaN
std,NaN,NaN,1.316894,1.24049,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000,1.00000,NaN,NaN,NaN,NaN
25%,NaN,NaN,2.000000,2.00000,NaN,NaN,NaN,NaN
50%,NaN,NaN,3.000000,3.00000,NaN,NaN,NaN,NaN
75%,NaN,NaN,4.000000,4.00000,NaN,NaN,NaN,NaN


### 🗑️ 8. Removal of Irrecoverable Records
- Dropped rows with missing Total Spent values, as they lacked essential financial information  
- This affected only a small fraction of the dataset and did not materially impact overall analysis  

In [43]:
df = df.dropna(subset=['Total Spent'])

### ✅ 9. Final Validation
- Reassessed null counts and overall completeness across all columns after cleaning  
- Confirmed that the dataset is now consistent, reliable, and ready for analysis  

In [45]:
df.isnull().sum()

Transaction ID       0
Item                 0
Quantity            18
Price Per Unit      18
Total Spent          0
Payment Method       0
Location             0
Transaction Date     0
dtype: int64

In [46]:
df.count()

Transaction ID      9960
Item                9960
Quantity            9942
Price Per Unit      9942
Total Spent         9960
Payment Method      9960
Location            9960
Transaction Date    9960
dtype: int64

- Exported the dataset to the Desktop as a CSV file named **“cleaned_cafe_sales”**

In [ ]:
df.to_csv('C:\Users\Anubhav\OneDrive\Desktop\Portfolio\cleaned_cafe_sales.csv', index=False)